In [22]:
import requests

from config import SPOT_CLIENT_SECRET, SPOT_CLIENT_ID
CLIENT_ID = SPOT_CLIENT_ID
CLIENT_SECRET = SPOT_CLIENT_SECRET


def get_spotify_token(client_id, client_secret):
    auth_response = requests.post(
        'https://accounts.spotify.com/api/token',
        {'grant_type': 'client_credentials'},
        auth=(client_id, client_secret)
    )
    return auth_response.json()['access_token']

def search_track(track_name, token):
    headers = {'Authorization': f'Bearer {token}'}
    search_params = {
        'q': track_name,
        'type': 'track',
        'limit': 1
    }
    response = requests.get('https://api.spotify.com/v1/search', headers=headers, params=search_params)
    return response.json()

# Substitua com suas credenciais do Spotify
client_id = CLIENT_ID
client_secret = CLIENT_SECRET

# Obter o token
token = get_spotify_token(client_id, client_secret)

# Buscar uma música
track_name = 'What Was I Made For?'
track_info = search_track(track_name, token)


In [23]:
def get_track_features(track_id, token):
    headers = {'Authorization': f'Bearer {token}'}
    response = requests.get(f'https://api.spotify.com/v1/audio-features/{track_id}', headers=headers)
    return response.json()

# Suponha que você tenha o ID da música do passo anterior
track_id = track_info['tracks']['items'][0]['id']
features = get_track_features(track_id, token)


In [24]:
print(features)

# Supondo que você tenha obtido os detalhes da faixa a partir da API
track_details = search_track(track_name, token)

# Certifique-se de extrair o nome e o artista corretamente
track_info = {
    'name': track_details['tracks']['items'][0]['name'],
    'artist': track_details['tracks']['items'][0]['artists'][0]['name'],
    # outras informações conforme necessário
}


{'danceability': 0.444, 'energy': 0.0911, 'key': 0, 'loudness': -17.665, 'mode': 1, 'speechiness': 0.0307, 'acousticness': 0.959, 'instrumentalness': 1.05e-06, 'liveness': 0.098, 'valence': 0.142, 'tempo': 78.403, 'type': 'audio_features', 'id': '6wf7Yu7cxBSPrRlWeSeK0Q', 'uri': 'spotify:track:6wf7Yu7cxBSPrRlWeSeK0Q', 'track_href': 'https://api.spotify.com/v1/tracks/6wf7Yu7cxBSPrRlWeSeK0Q', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6wf7Yu7cxBSPrRlWeSeK0Q', 'duration_ms': 222370, 'time_signature': 4}


In [25]:
# Suponha que você tenha o modelo carregado (por exemplo, modelo Random Forest)
import pandas as pd
import joblib

knn_model_loaded = joblib.load('KNN_MODEL.pkl')

# Preparar os dados

import pandas as pd
from sklearn.preprocessing import StandardScaler

# Supondo que 'features' é um dicionário com as características extraídas da API do Spotify
new_data = pd.DataFrame([features])

# Selecionar apenas as colunas relevantes
#relevant_columns = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
#new_data = new_data[relevant_columns]

# Selecione as colunas numéricas para normalização
relevant_columns = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
new_data = new_data[relevant_columns]

# Aplicando a normalização
scaler = StandardScaler()
new_data[relevant_columns] = scaler.fit_transform(new_data[relevant_columns])

# Aplicar a mesma normalização/padronização usada no treinamento
# Suponha que 'scaler' é o StandardScaler que você usou anteriormente
new_data_scaled = scaler.transform(new_data)

# Agora 'new_data_scaled' está pronto para ser usado para previsões


# Lembre-se de aplicar a mesma transformação que foi aplicada aos dados de treino
# Por exemplo, se usou StandardScaler, aplique-o aqui também
#data_to_predict = pd.DataFrame([features])
#prepared_data = scaler.transform(data_to_predict)

# Fazer a previsão
quadrant_prediction = knn_model_loaded.predict(new_data_scaled)


f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
f:\Programas\anaconda3\envs\uspspotenv\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version

In [26]:


quadrants = {
    'Q1': 'Alta Valência, Alta Ativação (Alegre, Energética)',
    'Q2': 'Baixa Valência, Alta Ativação (Tensa, Nervosa)',
    'Q3': 'Baixa Valência, Baixa Ativação (Triste, Melancólica)',
    'Q4': 'Alta Valência, Baixa Ativação (Calma, Relaxante)'
}

# Supondo que 'quadrants' é uma lista ou dicionário que mapeia os números da previsão para os rótulos dos quadrantes
predicted_quadrant = quadrants[quadrant_prediction[0]]

# Combinando as informações
track_info['predicted_quadrant'] = predicted_quadrant

# Exibindo as informações
print(f"Nome da Música: {track_info['name']}")
print(f"Artista: {track_info['artist']}")
print(f"Quadrante Previsto: {track_info['predicted_quadrant']}")
# ... exibir outras informações desejadas


Nome da Música: What Was I Made For? [From The Motion Picture "Barbie"]
Artista: Billie Eilish
Quadrante Previsto: Baixa Valência, Baixa Ativação (Triste, Melancólica)
